<a href="https://colab.research.google.com/github/muhammadibrohimov-ai/Deep-Learning/blob/main/classification_images_with_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

In [ ]:
# GPU borligini tekshirish
torch.cuda.is_available()

True

In [ ]:
# Torch ni GPU ga bog'lash
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
transforms = transforms.Compose([
    transforms.ToTensor()
])

In [11]:
train_data = torchvision.datasets.MNIST(
    root='./data',
    train=True,
    download=True,
    transform=transforms
)
test_data = torchvision.datasets.MNIST(
    root='./data',
    train=False,
    download=True,
    transform=transforms
)

100%|██████████| 9.91M/9.91M [00:01<00:00, 5.55MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 131kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.24MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 13.6MB/s]


In [12]:
train_loader = torch.utils.data.DataLoader(
    train_data,
    batch_size=64,
    shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    test_data,
    batch_size=64,
    shuffle=False
)

In [18]:
misol = next(iter(train_loader))
misol[0].shape

torch.Size([64, 1, 28, 28])

# Modelni yaratib olish

In [23]:
class OddiyCNN(nn.Module):
    def __init__(self):
        super(OddiyCNN, self).__init__()

        self.net = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Flatten(),
            nn.Linear(32*7*7, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        return self.net(x)


In [24]:
model = OddiyCNN().to(device)

In [25]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [28]:
epoxa = 10

losses = []

for epoch in range(epoxa):
    model.train()
    total_loss = 0

    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        # Forward jarayoni
        output = model(data)
        loss = criterion(output, target)

        # Backward jarayoni
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f'Epoxa: {epoch+1}, Loss: {total_loss/len(train_loader)}')

Epoxa: 1, Loss: 0.2203832107649715
Epoxa: 2, Loss: 0.0598833843214767
Epoxa: 3, Loss: 0.0420764146994716
Epoxa: 4, Loss: 0.0330029687664133
Epoxa: 5, Loss: 0.02560571440607746
Epoxa: 6, Loss: 0.02112230590633628
Epoxa: 7, Loss: 0.018222744065376724
Epoxa: 8, Loss: 0.012548073899734809
Epoxa: 9, Loss: 0.01291572773387472
Epoxa: 10, Loss: 0.01061793189623333


In [29]:
model.eval()

correct = 0
total = 0

with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)

        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

print(f'Test accuracy: {100*correct/total}%')

Test accuracy: 99.02%
